In [ ]:
import numpy as np
import hoomd 
import gsd.hoomd
import itertools

# Initial Configuration Generator

Use this script to generate the initial configuration of hard spheres at different sizes. The only thing you need to do is input the relevant parameters in the first cell. The notebook will do the rest. 

**Note: This script does not generate the correct volume of the box. Be sure to calculate the proper volume for the system and shrink the box properly during the simulation.**

In [ ]:
#input parameters 
init_config_filename =  #filename to create for the initial configuration

N_l =  #number of large particles
N_s =  #number of small particles

q =  #particle size ratio
dl =  #diameter of large colloid

ds = dl*q #diameter of small colloid
N_t = N_l+N_s #total number of particles

print(f'There are {N_t} particles in the system: \n\t{N_l} large particles with diameter {dl} \n\t{N_s} small particles with diameter {ds}.')

In [ ]:
spacing = dl+0.1
K = int(np.ceil(N_t**(1/3)))
L = K*spacing
x = np.linspace(-L/2, L/2, K, endpoint=False)
pos = list(itertools.product(x,repeat=3))

diameters = np.full(N_t,dl)
typeids = np.zeros(N_t)
types = ['large', 'small']

for i in range(N_s):
    typeids[i] = 1
    diameters[i] = ds
    
for i in range(N_l, N_t):
    j = np.random.randint(N_s)
    pos[j], pos[i] = pos[i], pos[j]
    
frame = gsd.hoomd.Frame()

frame.particles.N = N_t
frame.particles.position = pos[:N_t]
frame.particles.types = types
frame.particles.typeid = typeids
frame.particles.diameter = diameters
frame.configuration.box = [L, L, L, 0, 0, 0]

with gsd.hoomd.open(name=init_config_filename, mode='w') as f:
    f.append(frame)
f.close()